In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Recsys2022 Challenge 

NVIDIA-Merlin team participated in [Recsys2022 challenge](http://www.recsyschallenge.com/2022/index.html) and secured 3rd position. This notebook contains the various techniques used in the solution.

### Learning Objective
In this notebook, we will learn the importance of the concepts that improved the results of the competition significantly.

- ##### Label smoothing
     When the probabilities predicted by a Classification model are higher than its accuracy we say the model is overconfident. It can be prevented by using Label smoothing. This technique basically, transforms One-hot encoded labels into smoothed labels. 
$$  \begin{array}{l}
y_{l} \ =\ ( 1\ -\ \alpha \ ) \ *\ y_{o} \ +\ ( \alpha \ /\ L)\\
\alpha :\ Label\ smoothing\\
L:\ Total\ number\ of\ label\ classes\\
y_{o} :\ One-hot\ encoded\ label\ vector
\end{array}
$$
When α is 0, we have the original one-hot encoded labels, and as α increases, we move towards smoothed labels. Read [this](https://arxiv.org/abs/1906.02629) paper to learn more about it.


- ##### Temperature Scaling
    Similar to Label Smoothing, Temperature Scaling is done to reduce the overconfidence of a model. In this, we divide the logits (inputs to the softmax function) by a scalar parameter (T) . For more information on Temperature Scaling read [this](https://arxiv.org/pdf/1706.04599.pdf) paper.
$$ softmax\ =\ \frac{e\ ^{( z_{i} \ /\ \ T)}}{\sum _{j} \ e^{( z_{j} \ /\ T)} \ } $$


- ##### Weight Tying
In this technique, we share the Embedding layer's weights which is used to convert the input to embeddings, as the softmax weights,  to convert hidden layer output to softmax layer output. This drastically reduces the number of parameters and allows the model to train better. For more information read [this](https://arxiv.org/pdf/1608.05859v3.pdf) paper.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import cudf
import dask_cudf
import numpy as np
import pandas as pd 

import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.io import Dataset
from merlin.schema import Schema, Tags
from nvtabular.ops import (
    AddMetadata,
)
from merlin.schema.tags import Tags
from utils import get_drepessi_recsys2022_dataset


import tensorflow as tf
from merlin.io import Dataset
from merlin.schema import Tags
from tensorflow.keras import regularizers
from merlin.models.tf.dataset import BatchedDataset
from merlin.models.tf.utils.tf_utils import extract_topk

import merlin.models.tf as mm
from merlin.models.tf import InputBlock
from merlin.models.tf.models.base import Model
from merlin.models.tf.core.aggregation import SequenceAggregation, SequenceAggregator
from merlin.models.tf.core.transformations import (
    ItemsPredictionWeightTying,
    L2Norm,
    LogitsTemperatureScaler,
)


DATA_FOLDER = 'dressipi'
DATA_PROCESSED_FOLDER = 'dressipi_processed'
DATETIME_CONVERTION = 'ms'

2022-08-08 19:33:42.829651: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-08 19:33:44.824686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:0a:00.0, compute capability: 7.0


## Dressipi
The [Dressipi](http://www.recsyschallenge.com/2022/dataset.html) dataset contains 1.1 M online retail sessions that resulted in a purchase. It provides details about items that were viewed in a session, the item purchased at the end of the session and numerous features of those items. The task of this competition was, given a sequence of items predict which item will be purchased at the end of a session.


<img src="imgs/dressipi.JPG" alt="dressipi_dataset" style="width: 400px; float: center;">  


In [23]:
train, valid, sessions = get_drepessi_recsys2022_dataset(DATA_FOLDER)

## Feature Engineering with NVTabular

### Categorify

In [29]:
%%time
item_features_names = [col for col in train.columns if 'f_' in col]
cat_features = ['session_id', 'item_id', 'purchase_id'] + item_features_names >> nvt.ops.Categorify()

features = ['timestamp','date'] + cat_features
dataset = Dataset(sessions)
workflow0 = nvt.Workflow(features)
workflow0.fit(dataset)

# transform data
train_0 = workflow0.transform(Dataset(train))
valid_0 = workflow0.transform(Dataset(valid))

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 2.32 s, sys: 911 ms, total: 3.23 s
Wall time: 6.08 s


### GroupBy

In [30]:
%%time
features = train_0.head().columns.tolist()

# Define Groupby Operator
to_aggregate = {
    'date': ["first", "last"],
    'item_id': ["list"],
    'timestamp': ["list"],
    'purchase_id': ['first'],
}
for name in item_features_names: 
    to_aggregate[name] = ['list']
    
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    sort_cols=["date"],
    aggs= to_aggregate,
    name_sep="_")

# Add tags needed for the t4rec models definition
item_list = groupby_features['item_id_list'] >> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.ITEM_ID, Tags.LIST])
feature_list = groupby_features[[name+'_list' for name in item_features_names]]>> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.LIST])
other_features = groupby_features['session_id', 'date_first', 'date_last','timestamp_list']
target_feature = groupby_features['purchase_id_first'] >> nvt.ops.AddMetadata(tags=[Tags.TARGET])

workflow1 = nvt.Workflow(item_list + feature_list + other_features + target_feature)
workflow1.fit(train_0)

# transform data
train_1 = workflow1.transform(train_0)
valid_1 = workflow1.transform(valid_0)

CPU times: user 1.31 s, sys: 1.76 s, total: 3.07 s
Wall time: 3.15 s


### Truncate and Padding for a Maximum Sequence Length

In [32]:
SESSIONS_MAX_LENGTH = 20
list_cols = [col for col in train_1.head().columns if 'list' in col and 'date' not in col]
truncated_features = list_cols >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) >> nvt.ops.Rename(postfix = '_seq')

final_features = [
    'session_id', 'date_first', 'date_last', 'item_id_list', 'purchase_id_first'
]

workflow2 = nvt.Workflow(final_features + truncated_features)
workflow2.fit(train_1)

# transform data
train_2 = workflow2.transform(train_1)
valid_2 = workflow2.transform(valid_1)

### Save processed data to Parquet files

In [33]:
%%time
train_ds = Dataset(train_2.to_ddf().sort_values('date_last'), schema=train_2.schema)
valid_ds = Dataset(valid_2.to_ddf().sort_values('date_last'), schema=valid_2.schema)

train_ds.to_parquet(os.path.join(DATA_PROCESSED_FOLDER, "train/"), output_files=10)
valid_ds.to_parquet(os.path.join(DATA_PROCESSED_FOLDER, "valid/"), output_files=10)

CPU times: user 28.6 ms, sys: 4.34 ms, total: 32.9 ms
Wall time: 31.5 ms


## Training - MLP

A sequential-MLP model with average of the sequence as final representation

In [4]:
train = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'train/*.parquet'), shuffle=False,)
valid = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'valid/*.parquet'), shuffle=False,)

purchases = pd.read_csv(os.path.join(DATA_FOLDER, "train_purchases.csv"))
item_map = pd.read_parquet(
    os.path.join("categories", "unique.item_id.parquet"))['item_id'].to_dict()
session_map = pd.read_parquet(
    os.path.join("categories", "unique.session_id.parquet"))['session_id'].to_dict()

In [5]:
schema_model = train.schema.select_by_name(['item_id_list_seq', 'purchase_id_first'])
schema_model

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id_list_seq,"(Tags.LIST, Tags.ITEM, Tags.ITEM_ID, Tags.SEQU...",int64,True,False,None,0.0,0.0,0.0,.//categories/unique.item_id.parquet,23497.0,449.0,0,23497,item_id
1,purchase_id_first,"(Tags.TARGET, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,.//categories/unique.purchase_id.parquet,18908.0,397.0,0,18908,purchase_id


In [7]:
EPOCHS = 1
BATCH_SIZE = 512

### Model
InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [8]:
input_block = InputBlock(
        schema_model,
        aggregation='concat',
        seq=True,
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(embedding_dim_default=128),
        split_sparse=True,
)

MLPBlock to get the sequence of hidden representation

In [9]:
mlp_block = mm.MLPBlock(
                [64, 128],
                activation='relu',
                no_activation_last_layer=True,
                dropout=0.01,
            )

Multi-Classiffication Prediction head which has
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [10]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=2)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="purchase_id_first",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [11]:
model_mlp = Model(input_block, mlp_block, SequenceAggregator(SequenceAggregation.MEAN), prediction_task)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-1,
    clipnorm=True
)

# model_mlp.compile(optimizer=optimizer, run_eagerly=False)
model_mlp.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.2),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

2022-08-08 19:35:17.689787: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [36]:
model_mlp.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 parallel_block_3 (ParallelB  multiple                 3007744   
 lock)                                                           
                                                                 
 sequential_block_7 (Sequent  multiple                 16576     
 ialBlock)                                                       
                                                                 
 sequence_aggregator_2 (Sequ  multiple                 3007744   
 enceAggregator)                                                 
                                                                 
 purchase_id_first/multi_cla  (512, 23498)             3031242   
 ss_classification_task (Mul                                     
 tiClassClassificationTask)                                      
                                                           

### Model Training

In [ ]:
%%time
history = model_mlp.fit(
    train,
    validation_data=valid,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    schema=schema_model,
)

1422/1799 [======================>.......] - ETA: 31s - loss: 8.3850 - recall_at_100: 0.2304 - mrr_at_100: 0.0434 - ndcg_at_100: 0.0777 - map_at_100: 0.0434 - precision_at_100: 0.0023 - regularization_loss: 0.0000e+00

In [26]:
# def compute_mrr(rec_list,target):
#     mrr = 0
#     for a,b in zip(rec_list,target):
#         rank = np.argmax(np.array(a)==b)
#         if rank == 0 and a[0] == b:
#             mrr += 1
#         elif rank != 0:
#             mrr += (1 / (1 + rank))
#     return mrr/(target.shape[0])

In [27]:
# def model_evaluation(model):
#     valid = Dataset(
#     [DATA_PROCESSED_FOLDER+'/valid/*.parquet'], 
#     shuffle=False)
#     x = BatchedDataset(
#         valid, def compute_mrr(rec_list,target):
#     mrr = 0
#     for a,b in zip(rec_list,target):
#         rank = np.argmax(np.array(a)==b)
#         if rank == 0 and a[0] == b:
#             mrr += 1
#         elif rank != 0:
#             mrr += (1 / (1 + rank))
#     return mrr/(target.shape[0])
#         batch_size=512, 
#         shuffle=False, 
#     )
#     predictions = model.predict(x)

#     topk_predicted = []
#     for i in range(predictions.shape[0]):
#         _, topk_indices = tf.math.top_k(predictions[i, :], 100)
#         topk_predicted.append(topk_indices.numpy().reshape(1, 100))

#     top_predicted = np.concatenate(topk_predicted)

#     valid_data = valid.to_ddf().compute().to_pandas()
#     valid_data['session_id'] = valid_data.session_id.map(session_map)
#     valid_data = pd.merge(valid_data, purchases, on='session_id')[['session_id', 'item_id']]

#     valid_data['top100_predicted'] = top_predicted.tolist()
#     valid_data['top100_predicted']= valid_data['top100_predicted'].apply(lambda x: [item_map[i] for i in x])

#     return compute_mrr(valid_data['top100_predicted'], valid_data['item_id'])

### Model Evaluation

In [16]:
# %%time
# model_evaluation(model_mlp)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 1min 42s, sys: 12.4 s, total: 1min 55s
Wall time: 1min 30s


0.010065922109270606

In [ ]:
import cupy

def generate_recommendations(pred, df_agg, batch_size=1024, n_topk=100):
    print('Mask Predictions')
    print('Generate Top100 Recommendations')
    out_pred = []
    out_score = []
    for i in range(0, pred.shape[0]//batch_size+1):
        batch_start = (i)*batch_size
        batch_end = min((i+1)*batch_size, pred.shape[0])
        pred_tmp = pred[batch_start:batch_end]
        cp_pred = cupy.asarray(pred_tmp)
        pred_idx = cupy.argsort(-cp_pred)
        pred_idx = cupy.asnumpy(pred_idx)
        for j in range(pred_idx.shape[0]):
            topk = []
            score = []
            for k in range(n_topk):
                idx = pred_idx[j][k]
                topk.append(idx)
                score.append(pred_tmp[j][idx])
            out_pred.append(topk)
            out_score.append(score)
    
    print('Transform Top100 Recommendations')
    metadata = df_agg[['session_id', 'purchase_id_first']].to_pandas().values.tolist()
    out = []
    for i, ex in enumerate(metadata):
        session_id = ex[0]
        purchase = ex[1]
        for k in range(n_topk):
            out.append([session_id, purchase, out_pred[i][k], out_score[i][k]])

    df_rec = cudf.DataFrame(out)
    df_rec.columns = ['session_id', 'purchased', 'rec', 'score']
    return(df_rec)

def evaluate(df, add_folds=False):
    df = df.drop_duplicates(['session_id', 'rec'])
    df = df.sort_values(['session_id', 'score'], ascending=False)
    df['dummy'] = 1
    df['rank'] = df[['session_id', 'dummy']].groupby('session_id').cumsum()
    df = df[df['rank']<=100]
    df.drop('dummy', inplace=True, axis=1)
    df['mrr'] = 1/df['rank']
    df.loc[df['purchased']!=df['rec'], 'mrr'] = 0
    out = {}
    mrr = df[df['purchased']==df['rec']]['mrr'].sum()/df['session_id'].drop_duplicates().shape[0]
    out['total'] = mrr
    return(out)

predictions = model_mlp.predict(valid, batch_size=1024, verbose=1)
ddf = valid.to_ddf()
ddf = ddf[['session_id', 'purchase_id_first']].compute()
df_rec = generate_recommendations(predictions, ddf)
evaluate(df_rec)

## Training Bi-LSTM

In [17]:
EPOCHS = 10
BATCH_SIZE = 512
BI_LSTM_HIDDEN_DIM = 64

### Model

BiLSTM Block: It requires a dictionary input with the sequence of interaction embeddings `input_sequence`

In [18]:
class BiLSTM(mm.Block):
    def __init__(self, hidden_dim= 64, **kwargs):
        self.hidden_dim = hidden_dim
        lstm = tf.keras.layers.LSTM(hidden_dim, return_sequences=False, dropout=0.05,
                                   kernel_regularizer=regularizers.l2(1e-4))
        self.lstm = tf.keras.layers.Bidirectional(lstm)
        
        super().__init__(**kwargs)
        
    def call(self, inputs, training=False, **kwargs) -> tf.Tensor:  
        interactions = inputs['input_sequence']
        sequence_representation = self.lstm(interactions)
        return sequence_representation
    
    def compute_output_shape(self, input_shape):
        input_shape = input_shape['input_sequence']
        return (input_shape[0], input_shape[1], self.hidden_dim*2)
    
    
bilstm = BiLSTM(hidden_dim=BI_LSTM_HIDDEN_DIM)

InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [19]:
inputs = InputBlock(
        schema_model,
        aggregation='concat',
        seq=True,
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(
            embedding_dim_default=128,
            infer_embedding_sizes=True,
            infer_embedding_sizes_multiplier=2,
            infer_embeddings_ensure_dim_multiple_of_8=True
        ),
        split_sparse=True,
)

In [20]:
dense_block = mm.ParallelBlock({'input_sequence': inputs}).connect(bilstm)

MLPBlock to get the sequence of hidden representation

In [21]:
mlp_block = mm.MLPBlock(
                [64, 32],
                activation='relu',
                no_activation_last_layer=True,
                dropout=0.01,
            )

Multi-Classiffication Prediction head which has
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [22]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=2)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="purchase_id_first",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [23]:
model_bi_lstm = Model(dense_block, mlp_block, prediction_task)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-1,
    clipnorm=True
)

model_bi_lstm.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.2),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

### Model Training

In [24]:
%%time
history = model_bi_lstm.fit(train, validation_data=valid, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/10
1799/1799 [==============================] - 573s 312ms/step - loss: 13.4683 - recall_at_100: 0.1395 - mrr_at_100: 0.0177 - ndcg_at_100: 0.0389 - map_at_100: 0.0177 - precision_at_100: 0.0014 - regularization_loss: 4.5007 - val_loss: 14.7090 - val_recall_at_100: 0.1800 - val_mrr_at_100: 0.0201 - val_ndcg_at_100: 0.0483 - val_map_at_100: 0.0201 - val_precision_at_100: 0.0018 - val_regularization_loss: 5.5888
Epoch 2/10
1799/1799 [==============================] - 549s 302ms/step - loss: 14.9613 - recall_at_100: 0.1342 - mrr_at_100: 0.0173 - ndcg_at_100: 0.0377 - map_at_100: 0.0173 - precision_at_100: 0.0013 - regularization_loss: 5.9665 - val_loss: 14.6463 - val_recall_at_100: 0.1752 - val_mrr_at_100: 0.0184 - val_ndcg_at_100: 0.0463 - val_map_at_100: 0.0184 - val_precision_at_100: 0.0018 - val_regularization_loss: 5.5241
Epoch 3/10
1799/1799 [==============================] - 563s 310ms/step - loss: 20.7259 - recall_at_100: 0.1343 - mrr_at_100: 0.0173 - ndcg_at_100: 0.0378 -

### Model Evaluation

In [28]:
%%time
model_evaluation(model_bi_lstm)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 1min 46s, sys: 11.6 s, total: 1min 58s
Wall time: 1min 30s


0.010840330784918362